transorflow mask操作

In [1]:
import tensorflow as tf

In [2]:
# 定义输入
p = tf.constant([[2,3,4,0],[5,1,0,0],[4,1,0,0]])
h = tf.constant([[2,3,0,0],[6,3,5,0],[1,1,3,0]])


In [3]:
def length(sequence):
    """
    Get true length of sequences (without padding), and mask for true-length in max-length.

    Input of shape: (batch_size, max_seq_length, hidden_dim)
    Output shapes, 
    length: (batch_size)
    mask: (batch_size, max_seq_length, 1)
    """
    populated = tf.sign(tf.abs(sequence))
    length = tf.cast(tf.reduce_sum(populated, axis=1), tf.int32)
    mask = tf.cast(tf.expand_dims(populated, 1), tf.float32)

    # mask使为0的地方为0，其他地方为1
    return length, mask


In [4]:
l, m = length(h)
with tf.Session() as sess:
    print(sess.run(l))
    print(sess.run(m))

[2 3 3]
[[[1. 1. 0. 0.]]

 [[1. 1. 1. 0.]]

 [[1. 1. 1. 0.]]]


In [5]:
def masked_softmax(scores, mask):
    """
    Used to calculcate a softmax score with true sequence length (without padding), rather than max-sequence length.

    Input shape: (batch_size, max_seq_length, hidden_dim). 
    mask parameter: Tensor of shape (batch_size, max_seq_length). Such a mask is given by the length() function.
    """
    numerator = tf.exp(tf.subtract(scores, tf.reduce_max(scores, 1, keep_dims=True))) * mask
    denominator = tf.reduce_sum(numerator, 2, keep_dims=True)
    weights = tf.div(numerator, denominator)
    return weights

In [6]:
p_in = tf.constant([[[2,3],[5,1],[4,1],[0,0]],[[3,4],[5,1],[0,0],[0,0]],[[2,4],[2,1],[0,0],[0,0]]], dtype=tf.float32)

q_in = tf.constant([[[2,3],[5,1],[0,0],[0,0]],[[3,4],[5,1],[3,1],[0,0]],[[2,4],[2,1],[2,1],[0,0]]], dtype=tf.float32)

In [7]:
scores = tf.matmul(p_in, tf.transpose(q_in, [0,2,1]))
mask_scores = masked_softmax(scores, m)
p_hat = tf.matmul(mask_scores, q_in)
with tf.Session() as sess:
    print(sess.run(scores))
    print(sess.run(mask_scores))
    print(sess.run(p_hat))

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
[[[13. 13.  0.  0.]
  [13. 26.  0.  0.]
  [11. 21.  0.  0.]
  [ 0.  0.  0.  0.]]

 [[25. 19. 13.  0.]
  [19. 26. 16.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]]

 [[20.  8.  8.  0.]
  [ 8.  5.  5.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]]]
[[[9.9999774e-01 2.2603242e-06 0.0000000e+00 0.0000000e+00]
  [5.0000000e-01 5.0000000e-01 0.0000000e+00 0.0000000e+00]
  [9.5257413e-01 4.7425874e-02 0.0000000e+00 0.0000000e+00]
  [9.9999779e-01 2.2603244e-06 0.0000000e+00 0.0000000e+00]]

 [[9.5174748e-01 8.6788135e-04 4.7384717e-02 0.0000000e+00]
  [1.2378421e-03 4.9938112e-01 4.9938112e-01 0.0000000e+00]
  [1.2338896e-04 4.5392269e-05 9.9983120e-01 0.0000000e+00]
  [1.2338896e-04 4.5392269e-05 9.9983120e-01 0.0000000e+00]]

 [[3.3333334e-01 3.3333334e-01 3.3333334e-01 0

In [8]:
a = tf.constant(1e-13)
b = tf.constant(0.0)
c = tf.div(b,a)

with tf.Session() as sess:
    print(sess.run(c))

0.0


In [14]:
x = tf.constant([[[3,4,5],[4,5,6]],[[3,5,7],[8,9,5]]], dtype=tf.float32)
print(x.shape)
y = tf.constant([[[12], [15]],[[15], [22]]], dtype=tf.float32)
print(y.shape)

(2, 2, 3)
(2, 2, 1)


In [15]:
z = tf.div(x,y)
with tf.Session() as sess:
    print(sess.run(z))

[[[0.25       0.33333334 0.41666666]
  [0.26666668 0.33333334 0.4       ]]

 [[0.2        0.33333334 0.46666667]
  [0.36363637 0.4090909  0.22727273]]]
